In [1]:
# Load data from MySQL to perform exploratory data analysis
import settings
import mysql.connector
import pandas as pd
import time
import itertools
import math

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
#%matplotlib inline
# Using plotly.express
import plotly.express as px
import datetime
from IPython.display import clear_output

import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode()

In [2]:
'''
This plot shows the latest Twitter data within 30 mins and will automatically update.
In polarity:
    -1 means negative
    0  means neural
    +1 means positive
'''
while True:
    clear_output()
    db_connection = mysql.connector.connect(
        host="localhost",
        user="root",
        passwd="password",
        database="TwitterDB",
        charset = 'utf8'
    )
    # Load data from MySQL
    timenow = (datetime.datetime.utcnow() - datetime.timedelta(hours=0, minutes=10)).strftime('%Y-%m-%d %H:%M:%S')
    query = "SELECT id_str, text, created_at, polarity, user_location FROM {} WHERE created_at >= '{}' " \
                     .format(settings.TABLE_NAME, timenow)
    df = pd.read_sql(query, con=db_connection)
    # UTC for date time at default
    df['created_at'] = pd.to_datetime(df['created_at'])

    # Clean and transform data to enable time series
    result = df.groupby([pd.Grouper(key='created_at', freq='2s'), 'polarity']).count().unstack(fill_value=0).stack().reset_index()
    #result['created_at'] = pd.to_datetime(result['created_at']).apply(lambda x: x.strftime('%m-%d %H:%M'))
    #result = df
    result = result.rename(columns={"id_str": "Num of '{}' mentions".format(settings.TRACK_WORDS[0]), "created_at":"Time in UTC"})
    
    fig = px.line(result, x='Time in UTC', y="Num of '{}' mentions".format(settings.TRACK_WORDS[0]), color='polarity')
    fig.show()
    time.sleep(60)

KeyboardInterrupt: 

In [11]:
import re
import nltk
# nltk.download('punkt')
# nltk.download('stopwords')
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

db_connection = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="password",
    database="TwitterDB",
    charset = 'utf8'
)
#query = "SELECT id_str, text, created_at, polarity, user_location FROM {}  ".format(settings.TABLE_NAME)
timenow = (datetime.datetime.utcnow() - datetime.timedelta(hours=0, minutes=10)).strftime('%Y-%m-%d %H:%M:%S')
query = "SELECT id_str, text, created_at, polarity, user_location FROM {} WHERE created_at >= '{}' ".format(settings.TABLE_NAME, timenow)
    
df = pd.read_sql(query, con=db_connection)
# UTC for date time at default
df['created_at'] = pd.to_datetime(df['created_at'])
# 
'''
text = df.loc[:,['text']]
text['text'] = text['text'].str.replace('[^a-zA-Z]', ' ')
text['text'] = text['text'].str.replace('RT ', ' ')
text['text'] = text['text'].str.lower()
text['char_counts'] = text['text'].str.len()
text['word_counts'] = text['text'].str.split().str.len()
text['avg_word_len'] = text['char_counts'] / text['word_counts']
'''

content = ' '.join(df["text"])
content = re.sub(r"http\S+", "", content)
content = content.replace('RT ', ' ').replace('&amp;', 'and')
content = re.sub('[^A-Za-z0-9]+', ' ', content)
content = content.lower()

tokenized_word = word_tokenize(content)
stop_words=set(stopwords.words("english"))
filtered_sent=[]
for w in tokenized_word:
    if w not in stop_words:
        filtered_sent.append(w)
#print("Filterd Sentence:",filtered_sent)

fdist = FreqDist(filtered_sent)
fd = pd.DataFrame(fdist.most_common(25), columns = ["Word","Frequency"]).drop([0]).reindex()
# Plot Bar chart
fig = px.bar(fd, x="Word", y="Frequency")
fig.update_traces(marker_color='rgb(240,128,128)', marker_line_color='rgb(8,48,107)',
                  marker_line_width=1.5, opacity=0.8)
fig.show()

In [4]:
df.head()

id_str                                               text  \
0  1159842168971784198  RT @kjf1323: Kudos To All Trump Supporting Mom...   
1  1159842169080778752  RT @carlreiner: How can it be that one of the ...   
2  1159842168850145280  RT @BillyBobSanderz: So heres a list of compan...   
3  1159842169076645888  Does Trump realize that the briefing room in t...   
4  1159842169236049920  "Its like a physician diagnosing a sucking che...   

           created_at  polarity    user_location  
0 2019-08-09 15:01:36         0  Washington, USA  
1 2019-08-09 15:01:36         0             None  
2 2019-08-09 15:01:36         0            miami  
3 2019-08-09 15:01:36         0             None  
4 2019-08-09 15:01:36         0    United States

In [5]:
# Filter constants for US

STATES = ['Alabama', 'AL', 'Alaska', 'AK', 'American Samoa', 'AS', 'Arizona', 'AZ', 'Arkansas', 'AR', 'California', 'CA', 'Colorado', 'CO', 'Connecticut', 'CT', 'Delaware', 'DE', 'District of Columbia', 'DC', 'Federated States of Micronesia', 'FM', 'Florida', 'FL', 'Georgia', 'GA', 'Guam', 'GU', 'Hawaii', 'HI', 'Idaho', 'ID', 'Illinois', 'IL', 'Indiana', 'IN', 'Iowa', 'IA', 'Kansas', 'KS', 'Kentucky', 'KY', 'Louisiana', 'LA', 'Maine', 'ME', 'Marshall Islands', 'MH', 'Maryland', 'MD', 'Massachusetts', 'MA', 'Michigan', 'MI', 'Minnesota', 'MN', 'Mississippi', 'MS', 'Missouri', 'MO', 'Montana', 'MT', 'Nebraska', 'NE', 'Nevada', 'NV', 'New Hampshire', 'NH', 'New Jersey', 'NJ', 'New Mexico', 'NM', 'New York', 'NY', 'North Carolina', 'NC', 'North Dakota', 'ND', 'Northern Mariana Islands', 'MP', 'Ohio', 'OH', 'Oklahoma', 'OK', 'Oregon', 'OR', 'Palau', 'PW', 'Pennsylvania', 'PA', 'Puerto Rico', 'PR', 'Rhode Island', 'RI', 'South Carolina', 'SC', 'South Dakota', 'SD', 'Tennessee', 'TN', 'Texas', 'TX', 'Utah', 'UT', 'Vermont', 'VT', 'Virgin Islands', 'VI', 'Virginia', 'VA', 'Washington', 'WA', 'West Virginia', 'WV', 'Wisconsin', 'WI', 'Wyoming', 'WY']
STATE_DICT = dict(itertools.zip_longest(*[iter(STATES)] * 2, fillvalue=""))
INV_STATE_DICT = dict((v,k) for k,v in STATE_DICT.items())

In [6]:
#geo = df[['user_location']]
#geo = df[['user_location']][df['user_location'].apply(lambda x: x in (s for s in STATES))].reset_index(drop=True)
#geo.head(10)

In [7]:
is_in_US=[]
geo = df[['user_location']]
df = df.fillna(" ")
for x in df['user_location']:
    check = False
    for s in STATES:
        if s in x:
            is_in_US.append(STATE_DICT[s] if s in STATE_DICT else s)
            check = True
            break
    if not check:
        is_in_US.append(None)

#geo_dist = df[['user_location']][is_in_US]
geo_dist = pd.DataFrame(is_in_US, columns=['State']).dropna().reset_index()
geo_dist = geo_dist.groupby('State').count().rename(columns={"index": "Number"}) \
        .sort_values(by=['Number'], ascending=False).reset_index()
geo_dist["Log Num"] = geo_dist["Number"].apply(lambda x: math.log(x, 2))
geo_dist.head()

State  Number   Log Num
0    CA    1009  9.978710
1    TX     708  9.467606
2    NY     657  9.359750
3    FL     593  9.211888
4    IL     233  7.864186

In [9]:
import plotly.graph_objects as go

# Load data frame and tidy it.
import pandas as pd
colorscale = ["#f7fbff", "#ebf3fb", "#deebf7", "#d2e3f3", "#c6dbef", "#b3d2e9", "#9ecae1",
    "#85bcdb", "#6baed6", "#57a0ce", "#4292c6", "#3082be", "#2171b5", "#1361a9",
    "#08519c", "#0b4083", "#08306b"
]
geo_dist['Full State Name'] = geo_dist['State'].apply(lambda x: INV_STATE_DICT[x])
geo_dist['text'] = geo_dist['Full State Name'] + '<br>' + 'Num: ' + geo_dist['Number'].astype(str)
fig = go.Figure(data=go.Choropleth(
    locations=geo_dist['State'], # Spatial coordinates
    z = geo_dist['Log Num'].astype(float), # Data to be color-coded
    locationmode = 'USA-states', # set of locations match entries in `locations`
    colorscale = "Reds",
    text=geo_dist['text'], # hover text
    marker_line_color='white', # line markers between states
    colorbar_title = "Numbers in Log2"
))

fig.update_layout(
    title_text = "{} US '{}' mentions by State".format(datetime.datetime.utcnow().strftime('%Y-%m-%d %H:%M'), settings.TRACK_WORDS[0]),
    geo_scope='usa', # limite map scope to USA
)

fig.show()